In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install scikit-learn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.8 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 543.4 kB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.1/11.1 MB 2.8 MB/s eta 0:00:04
   - -------------------------------------- 0.3/11.1 MB 3.3 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/11.1 MB 4.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/11.1 MB 6.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.9/11.1 MB 7.9 MB/s eta 0:00:02
   ------- -------------------------------- 2.2/11.1 MB 7.6 MB/s eta 0:00:02
   --------- ------------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Load Data

In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Dataset Preparation
DATASET_DIR = "./dataset_destinasi"  # Direktori dataset
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
CLASSES = ["buildings", "forest", "glacier", "mountain", "sea", "street"]


In [10]:
# Transformasi Dataset
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [11]:
# Load Dataset
train_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, 'seg_train'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, 'seg_test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [12]:
# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [13]:
# Number of classes
num_classes = len(train_dataset.classes)
print(num_classes)

6


In [14]:
# # DataLoader
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=64,
#     shuffle=True,
#     num_workers=4
# )

# test_loader = DataLoader(
#     test_dataset,
#     batch_size=32,
#     shuffle=False,
#     num_workers=4
# )


# Model

## CNN

In [15]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

cnn_model = CNN(num_classes=len(CLASSES)).to(device)


# ResNet

In [16]:
# # Initialize Models
# num_classes = len(train_dataset.classes)
# cnn_model = CNN(num_classes).to(device)
# resnet_model = models.resnet18(weights='IMAGENET1K_V1')
# resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)
# resnet_model = resnet_model.to(device)

# # Loss and Optimizer
# criterion = nn.CrossEntropyLoss()
# cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
# resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

resnet_model = models.resnet18(pretrained=True)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, len(CLASSES))
resnet_model = resnet_model.to(device)


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
def train_and_validate(model, optimizer, train_loader, test_loader, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    train_loss_history, val_loss_history = [], []
    train_acc_history, val_acc_history = [], []

    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss, correct_train, total_train = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = 100 * correct_train / total_train
        train_loss /= len(train_loader)

        # Validation
        model.eval()
        val_loss, correct_val, total_val = 0, 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_accuracy = 100 * correct_val / total_val
        val_loss /= len(test_loader)

        # Logging
        train_loss_history.append(train_loss)
        val_loss_history.append(val_loss)
        train_acc_history.append(train_accuracy)
        val_acc_history.append(val_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    return train_loss_history, val_loss_history, train_acc_history, val_acc_history


In [18]:
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    print("Accuracy:", accuracy_score(all_labels, all_preds))
    print("Classification Report:\n", classification_report(all_labels, all_preds, target_names=CLASSES))


In [19]:
def plot_metrics(train_acc, val_acc, train_loss, val_loss):
    epochs = range(len(train_acc))
    plt.figure(figsize=(12, 5))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_acc, label='Training Accuracy')
    plt.plot(epochs, val_acc, label='Validation Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.show()


In [ ]:
# CNN Model Training
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
cnn_train_loss, cnn_val_loss, cnn_train_acc, cnn_val_acc = train_and_validate(cnn_model, cnn_optimizer, train_loader, test_loader)

evaluate_model(cnn_model, test_loader)
plot_metrics(cnn_train_acc, cnn_val_acc, cnn_train_loss, cnn_val_loss)
torch.save(cnn_model.state_dict(), "cnn_model.pth")

Epoch [1/10], Train Loss: 0.9317, Train Acc: 64.94%, Val Loss: 0.7351, Val Acc: 72.20%
Epoch [2/10], Train Loss: 0.6051, Train Acc: 78.26%, Val Loss: 0.5434, Val Acc: 80.70%
Epoch [3/10], Train Loss: 0.4732, Train Acc: 83.18%, Val Loss: 0.4963, Val Acc: 83.37%
Epoch [4/10], Train Loss: 0.3391, Train Acc: 87.79%, Val Loss: 0.4854, Val Acc: 84.43%
Epoch [5/10], Train Loss: 0.2086, Train Acc: 92.87%, Val Loss: 0.6528, Val Acc: 82.17%
Epoch [6/10], Train Loss: 0.1378, Train Acc: 95.30%, Val Loss: 0.6666, Val Acc: 82.63%


In [19]:
# ResNet Model Training
resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
resnet_train_loss, resnet_val_loss, resnet_train_acc, resnet_val_acc = train_and_validate(resnet_model, resnet_optimizer, train_loader, test_loader)

evaluate_model(resnet_model, test_loader)
plot_metrics(resnet_train_acc, resnet_val_acc, resnet_train_loss, resnet_val_loss)
torch.save(resnet_model.state_dict(), "resnet_model.pth")